## ETL -2

In [2]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('../datos/df_ccaa_year.csv', index_col=0)

In [3]:
df.sample(4)

,value,percentage,datetime,type,CCAA
80,3.618236e+06,0.193243,2013-01-01T00:00:00.000+01:00,renovable,Comunitat Valenciana
110,3.804298e+06,0.191425,2014-01-01T00:00:00.000+01:00,renovable,Comunitat Valenciana
99,1.014060e+07,0.296152,2014-01-01T00:00:00.000+01:00,no renovable,Castilla y León
144,1.513887e+07,0.515808,2015-01-01T00:00:00.000+01:00,renovable,Galicia


In [4]:
df_censal = pd.read_csv('../datos/poblacion_comunidades.csv', index_col=0)

In [5]:
df_censal.sample(4)

,Comunidad,indice,Comunidades_y_Ciudades_Autónomas,Año,Total
19,Ceuta,8744,Ceuta,2003,74.931
200,País Vasco,10,País Vasco,2022,2.208.174
26,Melilla,8745,Melilla,2021,86.261
202,País Vasco,10,País Vasco,2020,2.220.504


En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

El csv donde tenemos la información censal de cada Comunidad Autónoma tiene algunas columnas que no son redundantes para nuestro análisis, en este csv tendremos que:

Eliminar la columna de Comunidades_y_Ciudades_Autónomas

En el csv con la información de generación de energías renovables y no renovables a nivel nacional deberéis:
Para las columnas value y percentage, redondear los decimales de los valores a 2.
La columna datetime es de tipo string, pero esto a nosotras no nos interesa. Además, en esta columna tiene tanto la fecha como la hora, pero en realidad la hora no nos esta aportando nada de información, es por esto que:
Deberéis crear una columna nueva donde solo tengamos la información de la fecha en formato AÑO / MES / DIA.
Una vez creada la columna anterior, convertirla a datetime.

En el csv con la información de generación de energías renovables y no renovables a nivel de comunidad autónoma deberéis aplicar los mismos cambios que aplicamos en el ejercicio anterior.


In [6]:
#definimos función para borrar columnas
def borrar_columnas(df,col):
    df.drop(columns = col, axis = 1, inplace = True)
    return df

In [7]:
borrar_columnas(df_censal, "Comunidades_y_Ciudades_Autónomas")

,Comunidad,indice,Año,Total
0,Ceuta,8744,2022,83.117
1,Ceuta,8744,2021,83.517
2,Ceuta,8744,2020,84.202
3,Ceuta,8744,2019,84.777
4,Ceuta,8744,2018,85.144
...,...,...,...,...
470,La Rioja,20,2002,281.614
471,La Rioja,20,2001,270.400
472,La Rioja,20,2000,264.178
473,La Rioja,20,1999,265.178


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 359
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       360 non-null    float64
 1   percentage  360 non-null    float64
 2   datetime    360 non-null    object 
 3   type        360 non-null    object 
 4   CCAA        360 non-null    object 
dtypes: float64(2), object(3)
memory usage: 16.9+ KB


In [18]:
def limpiar_energia(df):
#esta parte de la función redondea las columnas numéricas
    for col in df.columns:
        if df[col].dtype == float:
            df[col] = round(df[col],2)
        else:
            pass
    return df

In [19]:
def cambiar_fecha(df,col):
#esta función modifica la fecha al formato solicitado y la deja con el tipo datetime
    df[col] = ((df[col].apply(pd.to_datetime)).dt.strftime("%Y-%m-%d")).apply(pd.to_datetime)
    return df

In [22]:
limpiar_energia(df)

,value,percentage,type,CCAA,fecha
0,10996921.24,0.28,renovable,Andalucía,2011-01-01
1,27608105.11,0.72,no renovable,Andalucía,2011-01-01
2,7005216.35,0.39,renovable,Aragón,2011-01-01
3,11119680.66,0.61,no renovable,Aragón,2011-01-01
4,338515.68,0.17,renovable,Cantabria,2011-01-01
...,...,...,...,...,...
355,7891030.21,0.35,no renovable,Galicia,2022-01-01
356,2670445.81,0.23,renovable,Región de Murcia,2022-01-01
357,8982495.46,0.77,no renovable,Región de Murcia,2022-01-01
358,1106679.16,0.48,renovable,La Rioja,2022-01-01


In [16]:
df2 = pd.read_csv('../datos/df_energia_nacional.csv', index_col = 0)
df2.sample(3)

,value,percentage,datetime,type
9,1.106052e+08,0.439959,2020-01-01T00:00:00.000+01:00,renovable
10,1.214591e+08,0.467130,2021-01-01T00:00:00.000+01:00,renovable
19,1.606375e+08,0.615512,2018-01-01T00:00:00.000+01:00,no renovable


In [20]:
limpiar_energia(df2)

,value,percentage,datetime,type
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,renovable
1,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,renovable
2,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,renovable
3,1.078761e+08,0.40,2014-01-01T00:00:00.000+01:00,renovable
4,9.408811e+07,0.35,2015-01-01T00:00:00.000+01:00,renovable
5,1.010892e+08,0.39,2016-01-01T00:00:00.000+01:00,renovable
6,8.461087e+07,0.32,2017-01-01T00:00:00.000+01:00,renovable
7,1.003444e+08,0.38,2018-01-01T00:00:00.000+01:00,renovable
8,9.791330e+07,0.38,2019-01-01T00:00:00.000+01:00,renovable
9,1.106052e+08,0.44,2020-01-01T00:00:00.000+01:00,renovable


In [21]:
cambiar_fecha(df2, "datetime")

,value,percentage,datetime,type
0,8.666705e+07,0.31,2011-01-01,renovable
1,8.533872e+07,0.30,2012-01-01,renovable
2,1.097577e+08,0.40,2013-01-01,renovable
3,1.078761e+08,0.40,2014-01-01,renovable
4,9.408811e+07,0.35,2015-01-01,renovable
5,1.010892e+08,0.39,2016-01-01,renovable
6,8.461087e+07,0.32,2017-01-01,renovable
7,1.003444e+08,0.38,2018-01-01,renovable
8,9.791330e+07,0.38,2019-01-01,renovable
9,1.106052e+08,0.44,2020-01-01,renovable


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   value       24 non-null     float64       
 1   percentage  24 non-null     float64       
 2   datetime    24 non-null     datetime64[ns]
 3   type        24 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 960.0+ bytes


In [23]:
df2.sample(3)

,value,percentage,datetime,type
1,8.533872e+07,0.30,2012-01-01,renovable
19,1.606375e+08,0.62,2018-01-01,no renovable
7,1.003444e+08,0.38,2018-01-01,renovable



Guardar los resultados obtenidos en pickle que usaremos en próximos ejercicios de pair programming. Cada fichero debe ir en un pickle

In [24]:
df2.to_csv('../datos/nacional_clean.csv')

In [29]:
df_censal.to_csv('../datos/poblacion_comunidades_clean.csv')

In [30]:
df.to_csv('../datos/df_ccaa_clean.csv')

In [25]:
df2.to_pickle('../datos/nacional_clean.pkl')

In [27]:
df3 = pd.read_csv('../datos/poblacion_comunidades_clean.csv')
df3.to_pickle('../datos/poblacion_comunidades_clean.pkl')

In [29]:
df4 = pd.read_csv('../datos/df_ccaa_clean.csv')
df4.to_pickle('../datos/df_ccaa_clean.pkl')
